In [1]:
#Data pre-processing
#Import data from csv file
import csv
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
#SVM 2 classes
#Create array matrix sizemxn where m is a number of data and n is a number of feature
vf_mat = np.zeros((121,53,))
vf_mat2 = np.zeros((121,55,))
with open('visual_field_pd.csv', newline='') as csvfile:
    vf_csv = csv.reader(csvfile)
    i = 0
    j = 0
    #Choose data by row
    for row in vf_csv:
        j=0
        #Choose single data in row
        for s in row:
            if int(row[0]) == 1:
                if j==0:
                    vf_mat[i,j] = 0
                else:
                    vf_mat[i,j] = int(s)
            elif int(row[0]) != 1:
                if j==0:
                    vf_mat[i,j] = 1
                else:
                    vf_mat[i,j] = int(s)                
            j=j+1
        i=i+1
        
with open('visual_field.csv', newline='') as csvfile:
    vf_csv = csv.reader(csvfile)
    i = 0
    j = 0
    #Choose data by row
    for row in vf_csv:
        j=0
        #Choose single data in row
        for s in row:
            vf_mat2[i,j] = int(s)
            j=j+1
        i=i+1

#Create array matrix of feature (Input)
X = vf_mat[:,1:]
X2 = vf_mat2[:,1:]

#Create array matrix of Output (Stage of Glaucoma 1 - 4)
y = vf_mat[:,0]
y2 = vf_mat2[:,0]-1
#Convert the Classification Report  into CSV file 
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-1] = total

    class_report_df['avg / total'] = avg
    return class_report_df.T

C:\Users\Maytus\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Maytus\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [10]:
#PCA
#Preprocessing data to standard variance
def pca_reduce_dim(X, dim):
    scaler = StandardScaler()
    scaler.fit(X)
    scaled_data = scaler.transform(X)
    #Applying PCA
    pca = PCA(n_components = dim)
    pca.fit(scaled_data)
    return(pca.transform(scaled_data))
X_pca = pca_reduce_dim(X2,2)

In [ ]:
n_rep = 5
c = 1
rkf = RepeatedKFold(n_splits=5, n_repeats=n_rep, random_state=None)
y_test_dict = dict()
y_test_dict2class = dict()
pred_dict = dict()
pred_dict2class = dict()
confusion_dict = dict()
confusion_dict2 = dict()
for train_index, test_index in rkf.split(X):
    #SVM1
    #Classify into 2 classes i.e. normal eye (0) and Glaucoma (1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y2[test_index]
    param_grid = {'C': [0.1,1,10,100,1000], 'gamma':[1,0.1,0.01,0.001,0.0001]}
    svc = GridSearchCV(SVC(), param_grid, verbose=1000)
    svc.fit(X_train, y_train)
    predictions = svc.predict(X_test)
    predictions2class = svc.predict(X_test)
    y_test2class = y[test_index]
    
    #SVM2
    #Classify Glaucoma into 3 stages (1-3)
    train_index2 = list()
    test_index2 = list()
    verify_index = list()
    for i in train_index:
        if y[i]!=0:
            train_index2.append(i)
    count_pred = 0
    for i in test_index:
        if predictions[count_pred] !=0:
            test_index2.append(i)
            verify_index.append(count_pred)
        count_pred+=1
    
    X2_train, X2_test = X_pca[train_index2], X_pca[test_index2]
    y2_train, y2_test = y2[train_index2], y2[test_index2]
    
    svc2 = GridSearchCV(SVC(), param_grid, verbose=1000)
    svc2.fit(X2_train, y2_train)
    predictions2 = svc2.predict(X2_test)
    
    predictions[verify_index] = predictions2
    #Keep y_test and predictions from each iteration in dict
    confusion_dict[str(c)] = pandas_classification_report(y_test,predictions)
    confusion_dict2[str(c)] = pandas_classification_report(y_test2class,predictions2class)
    
    #Keep y_test and predictions from each iteration in dict
    y_test_dict[str(c)] = y_test
    y_test_dict2class[str(c)] = y_test2class
    pred_dict[str(c)] = predictions
    pred_dict2class[str(c)] = predictions2class
    
    c+=1

In [7]:
pred_total = np.array([])
y_test_total = np.array([])
pred_total2class = np.array([])
y_test_total2class = np.array([])
for i in y_test_dict.values():
    y_test_total = np.concatenate((y_test_total,i),axis=0)
for i in pred_dict.values():
    pred_total = np.concatenate((pred_total,i),axis=0)
for i in y_test_dict2class.values():
    y_test_total2class = np.concatenate((y_test_total2class,i),axis=0)
for i in pred_dict2class.values():
    pred_total2class= np.concatenate((pred_total2class,i),axis=0)

In [8]:
#Classification
#0 = Normal Eye
#1 = Mild Glaucoma
#2 = Moderate Glaucoma
#3 = Advance Glaucoma
pandas_classification_report(y_test_total, pred_total)

,precision,recall,f1-score,support
0,0.925926,0.735294,0.819672,170.0
1,0.783410,0.918919,0.845771,185.0
2,0.956250,0.987097,0.971429,155.0
3,1.000000,0.978947,0.989362,95.0
avg / total,0.901747,0.894215,0.893178,605.0


In [9]:
#Detection
#0 = Normal Eye
#1 = Glaucoma
pandas_classification_report(y_test_total2class, pred_total2class)

,precision,recall,f1-score,support
0,0.925926,0.735294,0.819672,170.0
1,0.904255,0.977011,0.939227,435.0
avg / total,0.910345,0.909091,0.905633,605.0
